# Brad Evans: Flame-bait or Lame-bait?

[Blurb on why I'm doing this.]

## Methodology

Since Yahoo went with the infinite scrolling UI, pulling up old articles directly from the site became impossible. Thankfully there's Google. Using the following Google search, I can pull back every Flame/Lame call Brad has ever made:

site:sports.yahoo.com/blogs/fantasy-roto-arcade intitle:flames +2012 "Week 10"

site:sports.yahoo.com/blogs/fantasy-roto-arcade intitle:lames +2010 "Week 6"

There is a little manual work involved. Basically, I clicked to the last search result page, and clicked the link that said 

In order to show you the most relevant results, we have omitted some entries very similar to the 85 already displayed.
If you like, you can repeat the search with the omitted results included.

After passing all the captcha tests to prove to google and myself that I was human, I proceeded to Ctrl+A Ctrl+C each page of search results. Google obfuscates URLs, but thankfully, Yahoo's were just the right length for inferrance -

I also used Twitter and Internet Archive: 

flames OR lames @yahoonoise since:2011-01-01 until:2012-01-1

Basically, just use this regex in Vim, and you've got yourself a workable URL. 

:%s/\/\.\.\.\//\/blogs\/fantasy-roto-arcade\//



Yahoo's formats were terribly inconsistent. Bloated. 
Handling Updates.
Brad was a little more fearful early on (2010), suggesting a list of potential flames/lames in addition to his fearless forecasts.
Brad is mostly concerned with flames for 12-man leagues and deaper (prove this)
Under what conditions is Brad usually correct?

SHOCKER SPECIAL
BONUS SHOCKER (2011- week 2)
Attempting to fix missing data by padding with prefixes

Brad definitely didn't make this to be machine readable

A sample of formats (after heavily pre-processed) looks like this:

2009 
Josh Freeman, TB, QB (Noise QB Rank: 11, Start: 12-plus team leagues, 2% started)
Matchup: vs. NO
[WITTY BLURB]
Fearless Forecast: 19-33, 217 passing yards, 2 touchdowns, 1 interception, 24 rushing yards, 19 fantasy points

2010
Thomas Jones, KC, RB (Noise Week 11 RB Rank: 17, 53-percent started)
Matchup: vs. Ari
[WITTY BLURB]
Fearless Forecast: 19 carries, 95 rushing yards, 1 receptions, 4 receiving yards, 1 touchdown, 15 fantasy points

2011
Ben Tate, Hou, RB (Week 3 Noise RB Rank: 9, 22-percent started)
Matchup: at NO
[WITTY BLURB]
Fearless Forecast: 25 carries, 119 rushing yards, 4 receptions, 24 receiving yards, 1 touchdown, 23.3 fantasy points

2011
Michael Vick, Phi, QB (Noise Week 11 QB Rank: 13, 72-percent started)
Matchup: at NYG
[WITTY BLURB]
Fearless Forecast: 17-29, 193 passing yards, 1 touchdown, 2 interceptions, 27 rushing yards, 14.4 fantasy points

2012
Willis McGahee, Den, RB (78-percent started)
Matchup: vs. SD
[WITTY BLURB]
Fearless Forecast: 17 carries, 62 rushing yards, 3 receptions 21 receiving yards, 0 touchdowns, 10.5 fantasy points

2013
Matt Ryan, Atl, QB (53-percent started)
Matchup: at TB
[WITTY BLURB]
Fearless Forecast: 241 passing yards, 1 passing touchdown, 2 interceptions, 5 rushing yards, 14.5 fantasy points

2014
Ben Roethlisberger, Pit, QB (60 percent started)
Matchup: at Ten
[WITTY BLURB]
Fearless Forecast: 25-39, 266 passing yards, 1 passing touchdown, 1 interception, 16.3 fantasy points

2015
Matt Jones, Was, RB (10 percent started, $17)
Matchup: at NYG
[WITTY BLURB]
Fearless Forecast: 13 carries, 66 rushing yards, 3 receptions, 14 receiving yards, 1 touchdown, 15.5 fantasy points

This is fairly consistent now. The major difference going from 2009 to 2015, is in the rank. We start out with a "Noise Rank", and a league depth gague, but eventually just come down to a percent started, and this year, a dollar value added (for FanDuel, no doubt). 

Seeing as how this is Brad, I automatically assume it's geared toward AT LEAST 10 team leagues, but more likely 12. Basically, those in desparation. So I don't need that, nor do I need the out-of-context rank or the week, since we are given that. Dollar value could be interesting, if we were to extend this to daily leagues, so I'll keep that. 

Of course, none of this is easy, as there is no consistency. Sometimes he says the type of rank ("WR Rank") and others, just Rank. Sometimes the percentage is in numeric, other times he spells it out (two-percent). 

Lots of manual work here.

Then there's the Fearless Forecast part. Brad sometimes includes caveats "(Cutler or McCown), (minus Jones)" etc. So I've moved those up to the blurb. Sorry, Brad. Shit or get off the pot. And if you can't make a decision, I'll make one for you.

Year
Player_First_Name Player_Last_Name, Team_Abbreviation, Position (Percent_Started, [Daily_League_Dollar_Amt])
Matchup: [vs.|at] Team_Abbreviation
WITTY_BLURB
Fearless Forecast: [[0-9]*-[0-9]*,] 13 carries, 66 rushing yards, 3 receptions, 14 receiving yards, 1 touchdown, 15.5 fantasy points

Alright, let's start parsing.

In [119]:
import pprint
from pprint import pprint
import os
mypath = '/home/bdetweiler/src/Data Science/brad-evans-analysis/data'
from os import listdir
from os.path import isfile, join
from os import walk
for root, subdirs, files in os.walk(mypath):
    if 'preseason' in root: continue
    if 'free_agent' in root: continue
    for fileName in files:
        if not 'txt' in fileName: continue
        #if not '2014' in root: continue
        #if not 'week_03' in root: continue
        with open(root + '/' + fileName, 'r') as infile:
            print (root + '/' + fileName)
            data = infile.read()
            my_list = data.splitlines()
            date = my_list[0]
            player = my_list[2]
            shocker = False
            count = 0
            for line in my_list:
                if date in line: continue # we already have the date
                if '// Other' in line: continue 
                if 'SHOCKER' in line:
                    shocker = True
                    print('found a shocker in ' + root)
                    count = 1
                    continue
                    
                count += 1
                if count == 1: continue
                elif count == 2:
                    player = line
                elif count == 3:
                    matchup = line
                elif count == 4:
                    blurb = line
                elif count == 5:
                    fearless_forecast = line
                else:
                    count = 1
                    if shocker:
                        print ('----------------SHOCKER!--------------------------')
                        shocker = False
                    print('player: ' + player)
                    print('matchup: ' + matchup)
                    # print('blurb: ' + blurb)
                    print('fearless_forecast: ' + fearless_forecast)



/home/bdetweiler/src/Data Science/brad-evans-analysis/data/2009/week_07/flames.txt
player: Eli Manning, NYG, QB (45%)
matchup: Matchup: vs. Ari
fearless_forecast: Fearless Forecast: 21-34, 283 passing yards, 2 touchdowns, 1 interception, 21 fantasy points 
player: Jonathan Stewart, Car, RB (50%)
matchup: Matchup: vs. Buf
fearless_forecast: Fearless Forecast: 14 carries, 73 rushing yards, 1 reception, 7 receiving yards, 1 touchdown, 14 fantasy points
player: Laurence Maroney, NE, RB (34%)
matchup: Matchup: vs. TB (in London)
fearless_forecast: Fearless Forecast: 15 carries, 74 rushing yards, 2 receptions, 12 receiving yards, 1 touchdown, 14 fantasy points
player: Donald Brown, Ind, RB (25%)
matchup: Matchup: at StL
fearless_forecast: Fearless Forecast: 11 carries, 46 rushing yards, 2 receptions, 23 receiving yards, 1 touchdown, 12 fantasy points
player: Miles Austin, Dal, WR (47%)
matchup: Matchup: vs. Atl
fearless_forecast: Fearless Forecast: 5 receptions, 87 receiving yards, 1 touchdo